In [1]:
import sqlite3
import pandas as pd
import os

In [ ]:
# filtered_df = df[(df["year"]==2017) & (df["difficulty"]=="hard")]
# filtered_df

In [3]:
data = {
    'file_name': ['file1.docx', 'file2.docx', 'file3.docx', 'file4.docx'],
    'year': [2017, 2018, 2017, 2019],
    'difficulty': ['hard', 'easy', 'medium', 'hard'],
    'subject': ['math', 'science', 'math', 'english']
}

df = pd.DataFrame(data)
df

,file_name,year,difficulty,subject
0,file1.docx,2017,hard,math
1,file2.docx,2018,easy,science
2,file3.docx,2017,medium,math
3,file4.docx,2019,hard,english


In [5]:
df.to_csv("outputs/test_csv.csv", header=True, index=False)

In [6]:
read_df = pd.read_csv("csv_file/test_csv.csv")
read_df

,file_name,year,difficulty,subject
0,file1.docx,2017,hard,math
1,file2.docx,2018,easy,science
2,file3.docx,2017,medium,math
3,file4.docx,2019,hard,english


In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import pandas as pd

# 예시 MS Word 파일 경로
word_files = ["word_files/Q_1.docx", "word_files/Q_2.docx", "word_files/Q_3.docx", "word_files/Q_4.docx"]

# 체크박스 상태를 저장할 딕셔너리
checkbox_vars = {}

def show_filtered_files(filtered_files):
    """새 창에 필터링된 파일 목록을 체크박스로 표시."""
    selection_window = tk.Toplevel(root)
    selection_window.title("Select Files")
    selection_window.geometry("400x300")

    tk.Label(selection_window, text="Select Files to Process:").pack(pady=10)

    # 체크박스 생성
    for file in filtered_files:
        var = tk.IntVar()
        checkbox_vars[file] = var
        tk.Checkbutton(selection_window, text=file, variable=var).pack(anchor="w", padx=20)

    # 완료 버튼
    def submit_selection():
        selected_files = [file for file, var in checkbox_vars.items() if var.get() == 1]
        print(f"Selected Files: {selected_files}")
        selection_window.destroy()

    tk.Button(selection_window, text="Submit", command=submit_selection).pack(pady=10)

def apply_filter():
    """필터 적용 후 새 창에 결과 표시."""
    # 여기서는 예시로 모든 파일을 필터링된 결과로 사용
    filtered_files = word_files
    show_filtered_files(filtered_files)

# GUI 생성
root = tk.Tk()
root.title("File Filter")
root.geometry("500x500")
root.resizable(False, False)

# 필터 버튼
tk.Button(root, text="Filter and Show Files", command=apply_filter).pack(pady=20)

root.mainloop()


In [13]:
# 의존성 설치치
# !pip install python-docx pywin32 pymupdf pillow

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from win32com.client import Dispatch
import fitz  # PyMuPDF
from PIL import Image, ImageTk
import os

# 현재 실행 중인 디렉토리를 기준으로 기본 경로 설정
BASE_DIR = os.getcwd()
WORD_FILES_DIR = os.path.join(BASE_DIR, "word_files")

# 예시 MS Word 파일 경로
word_files = [
    os.path.join(WORD_FILES_DIR, "Q_1.docx"),
    os.path.join(WORD_FILES_DIR, "Q_2.docx"),
    os.path.join(WORD_FILES_DIR, "Q_3.docx"),
    os.path.join(WORD_FILES_DIR, "Q_4.docx"),
]

def word_to_pdf(docx_path):
    """Word 문서를 PDF로 변환."""
    pdf_path = docx_path.replace(".docx", ".pdf")
    word = Dispatch("Word.Application")
    doc = word.Documents.Open(docx_path)
    doc.SaveAs(pdf_path, FileFormat=17)  # 17은 PDF 포맷
    doc.Close()
    word.Quit()
    return pdf_path

def pdf_to_image(pdf_path):
    """PDF의 첫 페이지를 이미지로 변환."""
    pdf_document = fitz.open(pdf_path)
    page = pdf_document[0]  # 첫 페이지
    pix = page.get_pixmap(dpi=150)  # DPI를 설정해 해상도 조정
    image_path = pdf_path.replace(".pdf", ".png")
    pix.save(image_path)
    pdf_document.close()
    return image_path

def show_file_preview(file_path):
    """Word 파일의 첫 페이지를 이미지로 미리보기."""
    try:
        pdf_path = word_to_pdf(file_path)  # Word -> PDF
        image_path = pdf_to_image(pdf_path)  # PDF -> Image

        # 새 창에 이미지 표시
        preview_window = tk.Toplevel(root)
        preview_window.title(f"Preview: {file_path}")
        preview_window.geometry("600x800")

        img = Image.open(image_path)
        img = img.resize((500, 700), Image.Resampling.LANCZOS)  # 최신 Pillow 버전 대응
        photo = ImageTk.PhotoImage(img)

        label = tk.Label(preview_window, image=photo)
        label.image = photo  # 참조 유지
        label.pack(padx=10, pady=10)

        tk.Button(preview_window, text="Close", command=preview_window.destroy).pack(pady=10)

        # 임시 파일 삭제
        os.remove(pdf_path)
        os.remove(image_path)

    except Exception as e:
        print(f"Error: {e}")

def show_filtered_files(filtered_files):
    """새 창에 필터링된 파일 목록을 체크박스로 표시하고 미리보기 버튼 추가."""
    selection_window = tk.Toplevel(root)
    selection_window.title("Select and Preview Files")
    selection_window.geometry("500x400")

    tk.Label(selection_window, text="Select Files to Preview:").pack(pady=10)

    # 미리보기 버튼 생성
    for file in filtered_files:
        frame = tk.Frame(selection_window)
        frame.pack(anchor="w", padx=10, pady=2)

        tk.Button(frame, text=f"Preview {os.path.basename(file)}", command=lambda f=file: show_file_preview(f)).pack(side=tk.LEFT, padx=5)

    tk.Button(selection_window, text="Close", command=selection_window.destroy).pack(pady=10)

def apply_filter():
    """필터 적용 후 새 창에 결과 표시."""
    # 여기서는 예시로 모든 파일을 필터링된 결과로 사용
    filtered_files = word_files
    show_filtered_files(filtered_files)

# GUI 생성
root = tk.Tk()
root.title("File Filter and Preview")
root.geometry("500x500")
root.resizable(False, False)

# 필터 버튼
tk.Button(root, text="Filter and Show Files", command=apply_filter).pack(pady=20)

root.mainloop()

In [1]:
import tkinter as tk
from tkinter import ttk
from win32com.client import Dispatch
import fitz  # PyMuPDF
from PIL import Image, ImageTk
import os

# 현재 실행 중인 디렉토리를 기준으로 기본 경로 설정
BASE_DIR = os.getcwd()
WORD_FILES_DIR = os.path.join(BASE_DIR, "word_files")

# 예시 MS Word 파일 경로
word_files = [
    os.path.join(WORD_FILES_DIR, "Q_1.docx"),
    os.path.join(WORD_FILES_DIR, "Q_2.docx"),
    os.path.join(WORD_FILES_DIR, "Q_3.docx"),
    os.path.join(WORD_FILES_DIR, "Q_4.docx"),
]

def word_to_pdf(docx_path):
    """Word 문서를 PDF로 변환."""
    pdf_path = docx_path.replace(".docx", ".pdf")
    word = Dispatch("Word.Application")
    doc = word.Documents.Open(docx_path)
    doc.SaveAs(pdf_path, FileFormat=17)  # 17은 PDF 포맷
    doc.Close()
    word.Quit()
    return pdf_path

def pdf_to_image(pdf_path):
    """PDF의 첫 페이지를 이미지로 변환."""
    pdf_document = fitz.open(pdf_path)
    page = pdf_document[0]  # 첫 페이지
    pix = page.get_pixmap(dpi=150)  # DPI를 설정해 해상도 조정
    image_path = pdf_path.replace(".pdf", ".png")
    pix.save(image_path)
    pdf_document.close()
    return image_path

def show_filtered_images(filtered_files):
    """필터링된 파일을 이미지와 체크박스로 표시."""
    selection_window = tk.Toplevel(root)
    selection_window.title("Select Files with Preview")
    selection_window.geometry("800x600")

    # 스크롤 가능한 프레임
    canvas = tk.Canvas(selection_window)
    scrollbar = ttk.Scrollbar(selection_window, orient=tk.VERTICAL, command=canvas.yview)
    scrollable_frame = ttk.Frame(canvas)

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
    )

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)

    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    # 체크박스 상태 저장
    checkbox_vars = {}
    image_references = []  # 이미지 참조를 저장

    for file_path in filtered_files:
        frame = ttk.Frame(scrollable_frame)
        frame.pack(pady=10, padx=10, fill="x")

        var = tk.IntVar()
        checkbox_vars[file_path] = var

        try:
            pdf_path = word_to_pdf(file_path)
            image_path = pdf_to_image(pdf_path)

            # 미리보기 이미지 생성
            img = Image.open(image_path)
            img = img.resize((150, 200), Image.Resampling.LANCZOS)
            photo = ImageTk.PhotoImage(img)
            image_references.append(photo)  # 참조 유지

            # 이미지와 체크박스 추가
            tk.Label(frame, image=photo).pack(side="left", padx=5)
            tk.Checkbutton(frame, text=os.path.basename(file_path), variable=var).pack(side="left", padx=10)

            # 임시 파일 삭제
            os.remove(pdf_path)
            os.remove(image_path)

        except Exception as e:
            tk.Label(frame, text=f"Error loading {os.path.basename(file_path)}: {e}").pack()

    # 완료 버튼
    def submit_selection():
        selected_files = [file for file, var in checkbox_vars.items() if var.get() == 1]
        print(f"Selected Files: {selected_files}")
        selection_window.destroy()

    tk.Button(selection_window, text="Submit", command=submit_selection).pack(pady=10)

def apply_filter():
    """필터 적용 후 미리보기 이미지 표시."""
    # 여기서는 예시로 모든 파일을 필터링된 결과로 사용
    filtered_files = word_files
    show_filtered_images(filtered_files)

# GUI 생성
root = tk.Tk()
root.title("File Filter and Preview")
root.geometry("500x500")
root.resizable(False, False)

# 필터 버튼
tk.Button(root, text="Filter and Show Files", command=apply_filter).pack(pady=20)

root.mainloop()